In [1]:
from utils import *

In [2]:
# load data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/")

Extracting tmp/train-images-idx3-ubyte.gz
Extracting tmp/train-labels-idx1-ubyte.gz
Extracting tmp/t10k-images-idx3-ubyte.gz
Extracting tmp/t10k-labels-idx1-ubyte.gz


Once we've restored a graph, we need to figure out which ops to call to train it. Put those into active Python variables, and then continue as normal.

In [3]:
reset_graph()

saver = tf.train.import_meta_graph("savedmodels/11_07_gradientclipping.ckpt.meta")

Let's say we know that `X` and `y` are the inputs and that `accuracy` and `training_op` are needed to train and evaluate. That's all we need to extract.

In [4]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")
training_op = tf.get_default_graph().get_operation_by_name("train/GradientDescent")

**Note**: If you're making a nn intended for use by others, it's nice to group important operations into a collection and then telling people how to use it in the docs.

In [5]:
for op in (X, y, accuracy, training_op):
    tf.add_to_collection("important_ops", op)

In [6]:
# another person could load these ops as follows
X, y, accuracy, training_op = tf.get_collection("important_ops")

We've only restored the graph at this point. This is how we restore the weights, and pick up training where we left off.

In [7]:
n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    saver.restore(sess, "savedmodels/11_07_gradientclipping.ckpt")
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        if epoch % 5 == 0 or epoch == n_epochs - 1:
            acc_train = accuracy.eval(feed_dict={X:mnist.train.images, y:mnist.train.labels})
            acc_val = accuracy.eval(feed_dict={X:mnist.validation.images, y:mnist.validation.labels})
            print(epoch, "train acc:", acc_train, "val acc:", acc_val)
    acc_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
    print("Test acc:", acc_test)
    saver_path = saver.save(sess, "savedmodels/11_07_gradientclipping_contd.ckpt")

INFO:tensorflow:Restoring parameters from savedmodels/11_07_gradientclipping.ckpt
0 train acc: 0.964964 val acc: 0.9642
5 train acc: 0.971673 val acc: 0.9666
10 train acc: 0.975636 val acc: 0.9684
15 train acc: 0.981273 val acc: 0.9732
19 train acc: 0.984036 val acc: 0.9724
Test acc: 0.9705
